In [8]:
def calculate_coor(one_dim_arr):
    class_label = one_dim_arr[0]
    xcen = one_dim_arr[1]
    ycen = one_dim_arr[2]
    w = one_dim_arr[3]
    h = one_dim_arr[4]
    xmin = max(float(xcen) - float(w)/2, 0)
    xmax = min(float(xcen) + float(w)/2, 1)
    ymin = max(float(ycen) - float(h)/2, 0)
    ymax = min(float(ycen) + float(h)/2, 1)
    img_size_1 = 960
    img_size_0 = 720
    xmin = int(img_size_1 * xmin)
    xmax = int(img_size_1 * xmax)
    ymin = int(img_size_0 * ymin)
    ymax = int(img_size_0 * ymax)
    x_middle = (xmin + xmax)/2
    y_middle = (ymin + ymax)/2
    return x_middle, y_middle

In [9]:
from scipy.spatial import distance

def euclidean_dist(x1_middle, y1_middle, 
                   x2_middle, y2_middle):
    coor1 = [x1_middle, y1_middle]
    coor2 = [x2_middle, y2_middle]
    dst = distance.euclidean(coor1, coor2)
    return dst

In [10]:
## label content:
## 0~3:kidneys, 4:p0, 5:p1, 6:p2, 7:p3, 8:p4, 9:p5,
## 10:p6, 11:p7, 12:p8, 13:p9, 14:p10, 15:p11, 16:p12

In [11]:
from numpy import loadtxt
from operator import itemgetter
import numpy as np
import os

## Calculate the pixel distance between
## 2 pointers in each image & get unit pixel distance 
def min_max_unit_distance(text_source_dir):
    # get text files' paths
    text_files_list = os.listdir(text_source_dir)
    text_files_list.remove('classes.txt')
    text_name_list = []
    for text_name in text_files_list:
        if text_name.endswith('.txt'):
            text_name_list.append(text_name)    

    matchers = ['LT','RT']
    txt_name_matches = [s for s in text_name_list if any(
        xs in s for xs in matchers)]
    txt_name_matches = sorted(txt_name_matches)
    text_files_dir_list = [os.path.join(
        text_source_dir, name) for name in txt_name_matches]
            
    # load text files     
    i = 0# test
    pointer_distance_in_different_scale = []
    for text_file_dir in text_files_dir_list:
        i = i+1# test
        print(str(i)+':', text_file_dir)# test
        only_pointers_list = []
        txt_file = loadtxt(text_file_dir)
        for each_text_line in txt_file:
            # change label number to pointer number
            if each_text_line[0] > 3:
                each_line = each_text_line.tolist()
                if each_line[0] == 4: each_line[0] = 0
                if each_line[0] == 5: each_line[0] = 1
                if each_line[0] == 6: each_line[0] = 2
                if each_line[0] == 7: each_line[0] = 3
                if each_line[0] == 8: each_line[0] = 4
                if each_line[0] == 9: each_line[0] = 5
                if each_line[0] == 10: each_line[0] = 6
                if each_line[0] == 11: each_line[0] = 7
                if each_line[0] == 12: each_line[0] = 8
                if each_line[0] == 13: each_line[0] = 9
                if each_line[0] == 14: each_line[0] = 10
                if each_line[0] == 15: each_line[0] = 11
                if each_line[0] == 16: each_line[0] = 12
                only_pointers_list.append(each_line)        
        sort_pointers_list = sorted(
            only_pointers_list, key=itemgetter(0))
        sort_pointers_arr = np.array(sort_pointers_list)
        # calculate 2 pointers' x, y coordinates
        x1_middle, y1_middle = calculate_coor(
            sort_pointers_arr[0])
        x2_middle, y2_middle = calculate_coor(
            sort_pointers_arr[1])
        dst = euclidean_dist(x1_middle, y1_middle, 
                             x2_middle, y2_middle)
        # calculate pointer's unit distance 
        # for each image (e.g. p2-p0)  
        dif = sort_pointers_arr[1][0] - sort_pointers_arr[0][0]
        if dif == 1: unit_dst = dst
        if dif == 2: unit_dst = dst/2
        if dif == 3: unit_dst = dst/3
        if dif == 4: unit_dst = dst/4
        if dif == 5: unit_dst = dst/5
        if dif == 6: unit_dst = dst/6
        if dif == 8: unit_dst = dst/8
        if dif == 10: unit_dst = dst/10
        if dif == 12: unit_dst = dst/12
        pointer_distance_in_different_scale.append(unit_dst)
    # sanity check: 
    # check if text file number == len of pointer_dist_list
    if len(text_files_dir_list) != len(
        pointer_distance_in_different_scale):
        print("Calculation Error")
    # find max/min values in pointer's unit distance list
    min_distance = min(pointer_distance_in_different_scale)
    max_distance = max(pointer_distance_in_different_scale)
    
    return min_distance, max_distance, np.array(
        pointer_distance_in_different_scale), txt_name_matches

In [12]:
base_dir = '/home/ai03/_kunyi/pro_tasi/yolov5_5/datasets/'

In [13]:
text_source_dir = base_dir + "labels/no_UB"

In [14]:
train_min, train_max, train_all_dist, train_txt_list = min_max_unit_distance(text_source_dir)

1: /home/ai03/_kunyi/pro_tasi/yolov5_5/datasets/labels/no_UB/06236520_20121022_LT.txt
2: /home/ai03/_kunyi/pro_tasi/yolov5_5/datasets/labels/no_UB/06236520_20121022_RT.txt
3: /home/ai03/_kunyi/pro_tasi/yolov5_5/datasets/labels/no_UB/07956646_20141219_LT.txt
4: /home/ai03/_kunyi/pro_tasi/yolov5_5/datasets/labels/no_UB/07956646_20141219_RT.txt
5: /home/ai03/_kunyi/pro_tasi/yolov5_5/datasets/labels/no_UB/09850799_RT.txt
6: /home/ai03/_kunyi/pro_tasi/yolov5_5/datasets/labels/no_UB/10198502_20111102_LT.txt
7: /home/ai03/_kunyi/pro_tasi/yolov5_5/datasets/labels/no_UB/10198502_20111102_RT.txt
8: /home/ai03/_kunyi/pro_tasi/yolov5_5/datasets/labels/no_UB/10595700_20060412_LT.txt
9: /home/ai03/_kunyi/pro_tasi/yolov5_5/datasets/labels/no_UB/10595700_20060412_RT.txt
10: /home/ai03/_kunyi/pro_tasi/yolov5_5/datasets/labels/no_UB/11101099_20121224_LT.txt
11: /home/ai03/_kunyi/pro_tasi/yolov5_5/datasets/labels/no_UB/11101099_20121224_RT.txt
12: /home/ai03/_kunyi/pro_tasi/yolov5_5/datasets/labels/no_UB

302: /home/ai03/_kunyi/pro_tasi/yolov5_5/datasets/labels/no_UB/16730937_20151202_LT-L.txt
303: /home/ai03/_kunyi/pro_tasi/yolov5_5/datasets/labels/no_UB/16730937_20151202_RT-R.txt
304: /home/ai03/_kunyi/pro_tasi/yolov5_5/datasets/labels/no_UB/16732001_20150114_LT.txt
305: /home/ai03/_kunyi/pro_tasi/yolov5_5/datasets/labels/no_UB/16732001_20150114_RT.txt
306: /home/ai03/_kunyi/pro_tasi/yolov5_5/datasets/labels/no_UB/16732111_20150114_LT.txt
307: /home/ai03/_kunyi/pro_tasi/yolov5_5/datasets/labels/no_UB/16732111_20150114_RT.txt
308: /home/ai03/_kunyi/pro_tasi/yolov5_5/datasets/labels/no_UB/16732289_20150115_LT.txt
309: /home/ai03/_kunyi/pro_tasi/yolov5_5/datasets/labels/no_UB/16732289_20150115_RT.txt
310: /home/ai03/_kunyi/pro_tasi/yolov5_5/datasets/labels/no_UB/16734230_20160120_LT-R.txt
311: /home/ai03/_kunyi/pro_tasi/yolov5_5/datasets/labels/no_UB/16734230_20160120_RT-L.txt
312: /home/ai03/_kunyi/pro_tasi/yolov5_5/datasets/labels/no_UB/16737962_20150211_LT.txt
313: /home/ai03/_kunyi/p

In [15]:
from PIL import Image
from skimage.transform import resize

## Rescale all images in train, val, test test
## to same scale accoding to max unit distance
def scale_img(train_all_dist, txt_file_name_list, 
              img_source_dir, target_dir, 
              target_set_dist):

    all_set_distance = []
    all_set_distance.extend(train_all_dist)

    file_name_list = []
    for txt_name in txt_file_name_list:
        plur_file_name = txt_name[0:-4]
        file_name_list.append(plur_file_name)
    
    max_set_distance = max(all_set_distance)
    scale_factor_list = []
    for ori_dis in target_set_dist:
        scale_factor = max_set_distance / ori_dis
        scale_factor_list.append(scale_factor)

    img_files_list = os.listdir(img_source_dir)
    img_files_list = sorted(img_files_list)
    img_files_dir = [os.path.join(
        img_source_dir, name) for name in img_files_list]
    img_files_dir_list = []
    for img_file in img_files_dir:
        if img_file.endswith('.jpg'):
            img_files_dir_list.append(img_file)
    matchers = ['LT','RT']
    img_dir_matches = [s for s in img_files_dir_list if any(
        xs in s for xs in matchers)]
    
    for img_file, img_scale_factor, file_name in zip(
        img_dir_matches, scale_factor_list, file_name_list):

        img = Image.open(img_file)
        hsize = (float(img.size[1]) * img_scale_factor)
        wsize = (float(img.size[0]) * img_scale_factor)
        int_hsize = int(hsize)
        int_wsize = int(wsize)
        resized_img = img.resize((int_wsize, int_hsize))
        resized_img.save(target_dir+"/"+file_name+".jpg")
    
    return scale_factor_list, file_name_list

In [16]:
import pathlib

def check_if_file_exist(target_dir):
    file = pathlib.Path(target_dir)
    if file.exists():
        print ("File exist")
    else:
        print ("File not exist")
        os.makedirs(target_dir)

In [17]:
rescale_train_img_dir = base_dir + 'images/rescale_LtRt'
check_if_file_exist(rescale_train_img_dir)

File not exist


In [18]:
train_img_dir = base_dir + 'images/no_UB_crop'

In [19]:
train_scale_factor, train_img_name = scale_img(train_all_dist, 
    train_txt_list, train_img_dir, rescale_train_img_dir, 
    target_set_dist=train_all_dist)